In [ ]:
#completed output file and module name
#changed initial checks
#added earlyNewOutReady
#added NOIPS
#changes to HRR==1 mode

In [ ]:
#uses DotProduct_HR_#N_x#HRR_c#MAMCS_series - v4.2
#uses DotProduct_noHR_#N_c#MAMCS - v3.3

In [ ]:
import sys
import math

In [ ]:
#M>=1
M = input('Enter M: ')
M = int(M)

In [ ]:
#N>=1
#N better to be >= HRR, if N<HRR, can decrease HRR (to N)
N = input('Enter N: ')
N = int(N)

In [ ]:
#PR>=1
PR = input('Enter No Of Parallel Row: ')
PR = int(PR)

In [ ]:
#HRR>=1
HRR = input('Enter HRR: ')
HRR = int(HRR)

In [ ]:
if(N<1):
    N=1
if(HRR<1):
    HRR=1
NMult = math.ceil(N/HRR); #min Number of Multipliers Needed

In [ ]:
#>=NMult, <=N
if(HRR>1):
    NOIPS = input('Enter Number of inputs per series (>={}, <={}, empty for {}): '.format(NMult, N, NMult))

In [ ]:
#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size: ')
MAMCS = int(MAMCS)

In [ ]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value: ')
IN_WIDTH = int(IN_WIDTH)

In [ ]:
#>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value: ')
INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)

In [ ]:
#>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value: ')
MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)

In [ ]:
if(M<1):
    M=1
if PR<1:
    PR=1
elif PR>M:
    PR=M
RS=math.ceil(M/PR)

In [ ]:
if (RS>1):
    _Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')
else:
    _Enable_Vector_Latch = 0;

In [ ]:
if(HRR>1):
    if not NOIPS:
        NOIPS=NMult
    else:
        NOIPS = int(NOIPS)
        if(NOIPS<NMult):
            NOIPS=NMult
        else:
            if(NOIPS>N):
                NOIPS=N
            if NOIPS>NMult:
                NMult = NOIPS;
if MAMCS>NMult:
    MAMCS=NMult
if(IN_WIDTH<1):
    IN_WIDTH=1
if(INPUT_REG_DEPTH<0):
    INPUT_REG_DEPTH=0
if(MULT_PIPE_DEPTH<0):
    MULT_PIPE_DEPTH=0
if (RS>1):
    if not _Enable_Vector_Latch:
        _Enable_Vector_Latch = 0
    else:
        _Enable_Vector_Latch = int(_Enable_Vector_Latch)

In [ ]:
of=sys.stdout
# if(HRR==1):
#     if _Enable_Vector_Latch>0:
#         of=open("./matrix_vector_noHR_{}_{}_r{}_c{}_NIR_VL.v".format(M, N, PR, MAMCS), 'w+')
#     else:
#         of=open("./matrix_vector_noHR_{}_{}_r{}_c{}_NIR_NVL.v".format(M, N, PR, MAMCS), 'w+')
# else:
#     if _Enable_Vector_Latch>0:
#         of=open("./matrix_vector_{}_{}_r{}_x{}_p{}_c{}_series_NIR_VL.v".format(M, N, PR, HRR, NMult, MAMCS), 'w+')
#     else:
#         of=open("./matrix_vector_{}_{}_r{}_x{}_p{}_c{}_series_NIR_NVL.v".format(M, N, PR, HRR, NMult, MAMCS), 'w+')

In [ ]:
print("`timescale 1ns / 1ps\n", file=of)
if(HRR==1):
    if _Enable_Vector_Latch>0:
        print("module matrix_vector_noHR_{}_{}_r{}_c{}_NIR_VL".format(M, N, PR, MAMCS), file=of)
    else:
        print("module matrix_vector_noHR_{}_{}_r{}_c{}_NIR_NVL".format(M, N, PR, MAMCS), file=of)
else:
    if _Enable_Vector_Latch>0:
        print("module matrix_vector_{}_{}_r{}_x{}_p{}_c{}_series_NIR_VL".format(M, N, PR, HRR, NMult, MAMCS), file=of)
    else:
        print("module matrix_vector_{}_{}_r{}_x{}_p{}_c{}_series_NIR_NVL".format(M, N, PR, HRR, NMult, MAMCS), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
for j in range(PR):
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult):
        print("A{}r{}, ".format(i, j), end='', file=of)
    print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult):
    print("Vi{}, ".format(i), end='', file=of)
print(file=of)
if(HRR>1):
    print("output readyForNewRowStart,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output rowInSeries,", file=of)
elif lghrr>1:
    print("output [{}:0] rowInSeries,".format(lghrr-1), file=of)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("output reg columnInSeries = 0,", file=of)
    print("output reg outSeries = 1,", file=of)
else:
    print("output reg [{}:0] columnInSeries = 0,".format(lgRS-1), file=of)
    print("output reg [{}:0] outSeries = {},".format(lgRS-1, RS-1), file=of)
lgn=math.ceil(math.log2(N))
al=-2+lgn
if al==0:
    print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
elif al<0:
    print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
for i in range(PR):
    print("Vo{}, ".format(i), end='',file=of)
print("\noutput ", end='',file=of)
for i in range(PR):
    print("newVo{}Ready, ".format(i), end='', file=of)
print("\noutput ", end='',file=of)
for i in range(PR-1):
    print("earlyNewVo{}Ready, ".format(i), end='', file=of)
print("earlyNewVo{}Ready".format(PR-1), file=of)
print(");\n", file=of)

In [ ]:
if (RS>1) and (_Enable_Vector_Latch>0):
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult-1):
        print("SVi{}, ".format(i), end='', file=of)
    print("SVi{};".format(NMult-1), file=of)
    print(file=of)

In [ ]:
ar=RS*PR-M
if(ar>0): #RS*PR > M, RS>=2
    print("reg inReadyC;", file=of)
print("wire DPnewOutReady;", file=of)
if(ar>0):
    print("wire DPnewOutReadyC;", file=of)
print("wire DPearlyNewOutReady;", file=of)
if(ar>0):
    print("wire DPearlyNewOutReadyC;", file=of)
print(file=of)

for i in range (PR):
    if al==0:
        print("wire signed [(2*IN_WIDTH):0] DP{};".format(i), file=of)
    elif al<0:
        print("wire signed [(2*IN_WIDTH){}:0] DP{};".format(al, i), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DP{};".format(al, i), file=of)
    if(HRR==1):
        print("DotProduct_noHR_{}_c{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(N, MAMCS), file=of)
    else:
        print("DotProduct_HR_{}_x{}_p{}_c{}_series#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(N, HRR, NMult, MAMCS), file=of)
    print("\tDP_{}(clk, reset, enable,".format(i), file=of)
    if(PR-i<=ar):
        print("\tinReadyC,", file=of)
    else:
        print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(NMult):
        print("A{}r{}, ".format(j, i), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    if (RS>1) and (_Enable_Vector_Latch>0):
        for j in range(NMult):
            print("SVi{}, ".format(j), end='', file=of)
    else:
        for j in range(NMult):
            print("Vi{}, ".format(j), end='', file=of)        
    print(file=of)
    if(HRR>1):
        if i==0:
            print("\treadyForNewRowStart,", file=of)
            print("\trowInSeries,".format(i), file=of)
        else:
            print("\tLLreadyForNewDataSeries{}, //not used".format(i), file=of)
            print("\tLLinSeries{}, //not used".format(i), file=of)
    if i==0:
        print("\tDPnewOutReady,", file=of)
    elif i==PR-ar:
        print("\tDPnewOutReadyC,", file=of)
    else:
        print("\tDPnewOutReady{}, //not used".format(i), file=of)
    print("\tVo{},".format(i), file=of)
    if i==0:
        print("\tDPearlyNewOutReady\n\t);\n", file=of)
    elif i==PR-ar:
        print("\tDPearlyNewOutReadyC\n\t);\n", file=of)
    else:
        print("\tDPearlyNewOutReady{} //not used\n\t);\n".format(i), file=of)

In [ ]:
if(RS>1):
    if (_Enable_Vector_Latch>0):
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(N-1):
            print("LVi{}, ".format(i), end='', file=of)
        print("LVi{};".format(N-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tcolumnInSeries <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    if(HRR>1):
        print("\t\t\tif(rowInSeries=={}) begin".format(HRR-1), file=of)
        print("\t", end='', file=of)
    print("\t\t\tif(columnInSeries=={}) begin".format(RS-1), file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\t\tcolumnInSeries <= 0;", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\tend", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\telse begin", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    if RS==2:
        print("\t\t\t\tcolumnInSeries <= 1;", file=of)    
    else:
        print("\t\t\t\tcolumnInSeries <= columnInSeries+1;", file=of)
    if(HRR>1):
        print("\t\t\t\tend", file=of)
    print("\t\t\tend", file=of)
    if (RS>1) and (_Enable_Vector_Latch>0):
        print("\t\t\tif(columnInSeries==0) begin", file=of)
        if(HRR>1):
            print("\t\t\t\tcase(rowInSeries)", file=of)
        for i in range(HRR):
            if(HRR>1):
                print("\t\t\t\t\t{}: begin".format(i), file=of)
                print("\t\t", end='', file=of)
            print("\t\t\t\t", end='', file=of)
            k=NMult*i-1
            for j in range(NMult):
                k=k+1
                if k < N:
                    print("LVi{} <= Vi{}; ".format(k,j), end='', file=of)
            print(file=of)
            if(HRR>1):
                print("\t\t\t\t\tend", file=of)
            if k >= N-1:
                break
        if(HRR>1):
            print("\t\t\t\tendcase", file=of)
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [ ]:
if(RS>1):
    print("always @(*) begin", file=of)
    if (_Enable_Vector_Latch>0):
        print("\tif(columnInSeries==0) begin", file=of)
        print("\t\t", end='', file=of)
        for j in range(NMult):
            print("SVi{} = Vi{}; ".format(j,j), end='', file=of)
        print(file=of)
        print("\tend", file=of)
        print("\telse begin", file=of)
        if(HRR>1):
            print("\t\tcase(rowInSeries)", file=of)
        for i in range(HRR):
            if(HRR>1):
                print("\t\t\t{}: begin".format(i), file=of)
                print("\t\t", end='', file=of)
            print("\t\t", end='', file=of)
            k=NMult*i-1
            for j in range(NMult):
                k=k+1
                if k >= N:
                    print("SVi{} = 0; ".format(j), end='', file=of)            
                else:
                    print("SVi{} = LVi{}; ".format(j,k), end='', file=of)
            print(file=of)
            if(HRR>1):
                print("\t\t\tend", file=of)
        if(HRR>1):
            print("\t\tendcase", file=of)
        print("\tend", file=of)
    if(ar>0):
        print("\tif(columnInSeries=={}) begin".format(RS-1), file=of)
        print("\t\tinReadyC = 0;", file=of)
        print("\tend", file=of)
        print("\telse begin", file=of)
        print("\t\tinReadyC = inReady;", file=of)    
        print("\tend", file=of)
    print("end\n", file=of)

In [ ]:
if(RS>1):
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\toutSeries <= {};".format(RS-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(DPearlyNewOutReady) begin", file=of)
    print("\t\t\tif(outSeries=={}) begin".format(RS-1), file=of)
    print("\t\t\t\toutSeries <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\toutSeries <= 1;", file=of)
    else:
        print("\t\t\t\toutSeries <= outSeries+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [ ]:
for i in range(PR-ar):
    print("assign newVo{}Ready = DPnewOutReady;".format(i), file=of)
for i in range(PR-ar, PR):
    print("assign newVo{}Ready = DPnewOutReadyC;".format(i), file=of)
print(file=of)
for i in range(PR-ar):
    print("assign earlyNewVo{}Ready = DPearlyNewOutReady;".format(i), file=of)
for i in range(PR-ar, PR):
    print("assign earlyNewVo{}Ready = DPearlyNewOutReadyC;".format(i), file=of)
print(file=of)
print("endmodule", file=of)

In [ ]:
#of.close()